<a href="https://colab.research.google.com/github/dgo2dance/qwen-tune/blob/main/kuaishou_agent_fastapi_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [2]:
"""下载代码
"""
%cd /content/
!git lfs install
!git clone https://huggingface.co/kwaikeg/kagentlms_qwen_7b_mat_gguf


/content
Git LFS initialized.
Cloning into 'kagentlms_qwen_7b_mat_gguf'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 1), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (11/11), 2.58 KiB | 880.00 KiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	ggml-model-q4_0.gguf

See: `git lfs help smudge` for more details.


In [ ]:
!pip install llama_cpp_python[server]

In [ ]:
!pip3 install llama_cpp_python[server]

In [18]:
!pip -q install pyngrok
!pip install jiwer
!pip install tabulate
!pip install pydub
!pip -q install fastapi
!pip -q install pyngrok
!pip -q install uvicorn
!pip -q install nest_asyncio
!pip -q install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.4 MB/s eta 0:00:00


In [19]:
# Import libraries
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
from pydantic import BaseModel
import nest_asyncio

import shutil

nest_asyncio.apply()

In [20]:
# Import models for serialisation/ deserialisation
from pydantic import BaseModel
import base64
import io
import wave


class AudioBytes(BaseModel):
  data: bytes
  filename: str


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)


@app.get("/stt-test")
async def stt_test():
  return {"transcript":"Hello from STT."}

In [21]:
!ngrok config add-authtoken 2YvFOacMTFmP0jONlJAi7nUkbjr_48znoXDys5dh7qQDUHYyJ # place_your_ngrok_auth_token_here

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [36]:
# Create public URL to access Speech-To-Text service
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8090)
print("Public URL:", ngrok_tunnel.public_url)
# uvicorn.run(app, port=8000)

Public URL: https://ec15-34-69-232-70.ngrok-free.app


In [ ]:

uvicorn.run(app, port=8090)

In [37]:
!python3 -m llama_cpp.server --model /content/kagentlms_qwen_7b_mat_gguf/ggml-model-q4_0.gguf  --chat_format chatml --host 127.0.0.1 --port 8090

llama_model_loader: loaded meta data with 19 key-value pairs and 259 tensors from /content/kagentlms_qwen_7b_mat_gguf/ggml-model-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen
llama_model_loader: - kv   1:                               general.name str              = Qwen
llama_model_loader: - kv   2:                        qwen.context_length u32              = 8192
llama_model_loader: - kv   3:                           qwen.block_count u32              = 32
llama_model_loader: - kv   4:                      qwen.embedding_length u32              = 4096
llama_model_loader: - kv   5:                   qwen.feed_forward_length u32              = 22016
llama_model_loader: - kv   6:                        qwen.rope.freq_base f32              = 10000.000000
llama_model_loader: - kv   7:                  qwe

INFO:     Shutting down
INFO:     Finished server process [54809]
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 404, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/cors.py", line 91, in __c

In [28]:
print('查看内存容量')
!cat /proc/meminfo | grep MemTotal


查看内存容量
MemTotal:       13290480 kB


In [10]:
! lsof -i -n

COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
node        6 root   21u  IPv6  18762      0t0  TCP *:8080 (LISTEN)
node        6 root   27u  IPv4  48461      0t0  TCP 172.28.0.12:41950->172.28.0.12:6000 (ESTABLISHED)
node        6 root   28u  IPv6 107237      0t0  TCP 172.28.0.12:8080->172.28.0.1:54314 (ESTABLISHED)
node        6 root   29u  IPv6 103733      0t0  TCP 172.28.0.12:8080->172.28.0.1:47402 (ESTABLISHED)
node        6 root   34u  IPv6 104794      0t0  TCP 172.28.0.12:8080->172.28.0.1:46602 (ESTABLISHED)
node        6 root   36u  IPv6 106845      0t0  TCP 172.28.0.12:8080->172.28.0.1:51054 (ESTABLISHED)
kernel_ma  21 root    3u  IPv4  18268      0t0  TCP 172.28.0.12:6000 (LISTEN)
kernel_ma  21 root    7u  IPv4  48111      0t0  TCP 172.28.0.12:6000->172.28.0.12:41950 (ESTABLISHED)
kernel_ma  21 root    8u  IPv4  19745      0t0  TCP 172.28.0.12:36772->172.28.0.12:9000 (ESTABLISHED)
kernel_ma  21 root   10u  IPv4  23334      0t0  TCP 172.28.0.12:6000->172.28.0.1:48564 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp /content/kagentlms_qwen_7b_mat_gguf/ggml-model-q4_0.gguf   "/content/drive/My Drive/"